In [1]:
import pandas as pd
import numpy as np
import shutil
import os
import datetime

elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\CSIRO_GBR_ElementMapper.csv'
#Names from element mapper
regID = 'RegID'
riverCol = 'Major River'
elementCol = 'Network Element'

simMapperCSV = r'P:\projects\eReefs\GBRF_SpeedScenarios\GBRF_ScenarioExtension_ResultsMapperFiltered.csv'
#Names from sim mapper, use RegID from above
basinFolderName = 'BasinFolder'
newSimFolderName = 'SIMCODE'
#Switch these for TS Dependent or EMCMonthly
existSimFolderName = 'TSDependentSubDir'
#existSimFolderName = 'EMCMonthly2021SubDir'

#Switch these for parent location of TS Dependent or EMCMonthly results
#TSDependent
parentResultsDir = r'E:\GBRFScenarios\Results'
addRegIdDIR = True
outPath = r'E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet'
#Empirical
#parentResultsDir = r'P:\projects\RC10_ResultsSets\Models\results'
# addRegIdDIR = False
# outPath = r'E:\GBRFScenarios\CSIROAggregatedOutlets\EmpiricalEMC'

#, 'Sediment - Coarse'

Constituents = ['Sediment - Fine', 'N_Particulate', 'N_DIN', 'N_DON', 'P_Particulate', 'P_DOP', 'P_FRP']
#Constituents = ['Ametryn', 'S-metolachlor', 'Atrazine', 'Diuron', 'Hexazinone', 'Tebuthiuron', '24-D'
#                , 'Paraquat', 'Glyphosate', 'Chlorsulfuron', 'Diquat', 'Fluroxypyr', 'Haloxyfop', 'Imazapic'
#               , 'Imazethapyr', 'Isoxaflutole', 'MCPA', 'Metribuzin', 'Metsulfuron-methyl', 'Pendimethalin', 'Simazine'
#               , 'Terbuthylazin', 'Trifluralin', 'Aciflurofen', 'Chlorpyrifos', 'Fipronil', 'Imidacloprid', 'Prometryn', 'Triclopyr']

#regionIDs = ['BU','FI','CY','BM', 'WT', 'MW']
#,'CY':'CY Rebuild 2015'
#regionIDs = {'BU':'Burdekin Rebuild 2014'}
regionIDs = {'BM':'Burnett Mary Rebuild 2014','BU':'Burdekin Rebuild 2014','FI':'FI RC2019',
              'MW':'MW_RC10', 'WT':'WT_RC10'}

#elementMapperCSV = r'P:\projects\eReefsOperational\CSIRO_Locs\BurdekinElementMapper.csv'
#mainPath = r'P:\projects\MichaelWarne\Models\results\Burdekin Rebuild 2014\BU_msPAF_Calibrated_503\TimeSeries'
#outPath = r'C:\DDrive\RC10PesticideProjs\CSIROAggregatedOutlets\Burdekin'



In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:

# ##This bit now needs to happen inside of loop
# RiverElements = {}
# elementsTable = pd.read_csv(elementMapperCSV)

# for index, row in elementsTable.iterrows():
#     if not row[riverCol] in RiverElements.keys():
#         RiverElements[row[riverCol]] = []
    
#     RiverElements[row[riverCol]].append(row[elementCol])


# #print(RiverElements)

In [4]:
allElementsTable = pd.read_csv(elementMapperCSV)

In [5]:
simDetailsDF = pd.read_csv(simMapperCSV)

In [6]:
def getResultsPathInfo(regionIDString, scenarioString):
    tsPath = ''
    if addRegIdDIR:
        tsPath = parentResultsDir + '\\' + regionIDString + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    else:
        tsPath = parentResultsDir + '\\' + regionIDs[regionIDString] + '\\' + scenarioString + '\\TimeSeries'
    return tsPath

In [7]:
def doThisSim(thisRegID, thisBasinFolderName, thisNewSimFolderName, thisExistSimFolderName, theseRiverElements):
    
    #Going to need to set up folders
    #basinOutputStr = outPath + '\\' + thisBasinFolderName
    basinOutputStr = os.path.join(outPath, thisBasinFolderName)
    if not os.path.exists(basinOutputStr):
        os.makedirs(basinOutputStr)
        print("Made dir: " + basinOutputStr)
    
    
    simOutputStr = os.path.join(basinOutputStr, thisNewSimFolderName)
    if not os.path.exists(simOutputStr):
        os.makedirs(simOutputStr)
        print("Made dir: " + simOutputStr)    
    
    #Set up the totals first
    RiverElements = {}
    # elementsTable = pd.read_csv(elementMapperCSV)

    for index, row in theseRiverElements.iterrows():
         if not row[riverCol] in RiverElements.keys():
            RiverElements[row[riverCol]] = []
    
         RiverElements[row[riverCol]].append(row[elementCol])
    
    riverTotals = {}
    for theRiver in RiverElements.keys():
        riverTotals[theRiver] = pd.DataFrame()

    #Now data
    mainPath = getResultsPathInfo(thisRegID, thisExistSimFolderName)

    theFlowsDIR = os.path.join(mainPath,'Flows')
    for theRiver in RiverElements.keys():
        #We'll want flows first
        eleCount = 0
        flowFrame = pd.DataFrame()
        outputFields = ['Date']
        for theElement in RiverElements[theRiver]:
            for fname in os.listdir(theFlowsDIR):
                if theElement in fname:
                    #print("For this river: " + theRiver + " we have this file: " + fname)
                    flowFilePath = os.path.join(theFlowsDIR,fname)
                    flowData = pd.read_csv(flowFilePath)
                    flowData.columns = ['Date', theElement]
                    #riverTotals[theRiver].columns.drop(theElement)
                    outputFields.append(theElement)
                    if eleCount == 0:
                        #Get the structure for our totals first
                        riverTotals[theRiver] = flowData
                        riverTotals[theRiver].drop(theElement, axis=1, inplace=True)
                        #Now process stuff
                        flowFrame = flowData
                        #flowFrame.columns = ['Date', theElement]
                        #flowFrame['Date'] = flowData['Date']
                        #flowFrame[theElement] = flowData.iloc[:1].values#This sould give the 2nd column
                    else:
                        #Join...??
                        #flowFrame[theElement] = flowData.iloc[:1].values
                        flowFrame = pd.merge(flowFrame, flowData, how='left', left_on=['Date'], right_on=['Date'])
                
                    eleCount += 1
        #Totals
        #The .iloc[:, 1:] should exclude our first col, Date, from the sum
        #flowFrame['Flow_cumecs'] = flowFrame.iloc[:, 1:].sum(axis=1)
        flowFrame['Flow_cumecs'] = flowFrame.sum(axis=1)
        #Write out flow
        theOutFlowFile = os.path.join(simOutputStr, "Flow " + theRiver + " cubicmetrespersecond.csv")
        flowFrame.to_csv(theOutFlowFile, index=False)
        #Need to join flow Total to RiverTotals
        riverTotals[theRiver] = pd.merge(riverTotals[theRiver], flowFrame[['Date', 'Flow_cumecs']], how='left', left_on=['Date'], right_on=['Date'])

    #Now water quality
    for theWQBit in Constituents:
        wqPath = os.path.join(mainPath, theWQBit)
        if not os.path.exists(wqPath):
            continue
    
        for theRiver in RiverElements.keys():
            #We'll want flows first
            eleCount = 0
            wqFrame = pd.DataFrame()
            outputFields = ['Date']
            for theElement in RiverElements[theRiver]:
                for fname in os.listdir(wqPath):
                    if theElement in fname:
                        #print("For this river: " + theRiver + " we have this file: " + fname)
                        wqFilePath = os.path.join(wqPath,fname)
                        wqData = pd.read_csv(wqFilePath)
                        wqData.columns = ['Date', theElement]
                        outputFields.append(theElement)
                        if eleCount == 0:
                            wqFrame = wqData
                        else:
                            wqFrame = pd.merge(wqFrame, wqData, how='left', left_on=['Date'], right_on=['Date'])
                
                        eleCount += 1
            #Totals
            #wqFrame[theRiver + '_kg'] = wqFrame.iloc[:, 1:].sum(axis=1)
            wqFrame[theRiver + '_kg'] = wqFrame.sum(axis=1)
            #Write out flow
            theOutFlowFile = os.path.join(simOutputStr, theWQBit + " " + theRiver + " kilograms.csv")
            wqFrame.to_csv(theOutFlowFile, index=False)
            #Join to totals
            #rename constituent total first
            #print('theRiver: ' + theRiver)
            #print('theWQBit: ' + theWQBit)
            wqFrame.rename(columns={theRiver + '_kg':theWQBit + '_kg'}, inplace=True)
            riverTotals[theRiver] = pd.merge(riverTotals[theRiver], wqFrame[['Date', theWQBit + '_kg']], how='left', left_on=['Date'], right_on=['Date'])


    #Now write totals files
    for theRiver in RiverElements.keys():
        theOutTotalsFile = os.path.join(simOutputStr, theRiver + " Totals.csv")
        riverTotals[theRiver].to_csv(theOutTotalsFile, index=False)


    print("Done " + thisNewSimFolderName + " at " + str(datetime.datetime.now()))

In [8]:
#This is where we will loop through sims
for index, row in simDetailsDF.iterrows():
    if row[regID] in regionIDs.keys():
        print("Doing this simulation: " + row[newSimFolderName] + " at " + str(datetime.datetime.now()))
        #Filter out the river elements for this region
        filteredElements = allElementsTable.loc[allElementsTable[regID] == row[regID]]
        doThisSim(row[regID], row[basinFolderName], row[newSimFolderName], row[existSimFolderName], filteredElements)

print("All finished at " + str(datetime.datetime.now()))

Doing this simulation: BOWBUSLENOPASS at 2021-10-01 12:50:00.285443


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:100: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


Done BOWBUSLENOPASS at 2021-10-01 12:50:20.799549
Doing this simulation: BOWBUSLEWPASS at 2021-10-01 12:50:20.809676
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\BOWBUSLEWPASS
Done BOWBUSLEWPASS at 2021-10-01 12:50:39.274464
Doing this simulation: BOWBGULLY1 at 2021-10-01 12:50:39.282510
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\BOWBGULLY1
Done BOWBGULLY1 at 2021-10-01 12:50:57.526920
Doing this simulation: BOWBGULLY3 at 2021-10-01 12:50:57.526920
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\BOWBGULLY3
Done BOWBGULLY3 at 2021-10-01 12:51:15.997087
Doing this simulation: EASBUSLENOPASS at 2021-10-01 12:51:15.999136
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\EASBUSLENOPASS
Done EASBUSLENOPASS at 2021-10-01 12:51:36.483334
Doing this simulation: EASBUSLEWPASS at 2021-10-01 12:51:36.485367
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\EASBUSLEWPASS
Done

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node4_x'} in the result is deprecated and will raise a MergeError in a future version.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node1_x'} in the result is deprecated and will raise a MergeError in a future version.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node4_x'} in the result is deprecated and will raise a MergeError in a future version.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node1_x'} in the result is deprecated and will raise a MergeError in a future version.


Done HERBUSLENOPASS at 2021-10-01 12:54:22.894677
Doing this simulation: HERBUSLEWPASS at 2021-10-01 12:54:22.902718
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\WetTropics\HERBUSLEWPASS
Done HERBUSLEWPASS at 2021-10-01 12:54:45.921300
Doing this simulation: HERBBANK at 2021-10-01 12:54:45.923330
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\WetTropics\HERBBANK
Done HERBBANK at 2021-10-01 12:55:08.619467
Doing this simulation: MARYBANK at 2021-10-01 12:55:08.629577
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\BurnettMary
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\BurnettMary\MARYBANK
Done MARYBANK at 2021-10-01 12:55:20.741778
Doing this simulation: LOWBFERTDB at 2021-10-01 12:55:20.749845
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\LOWBFERTDB
Done LOWBFERTDB at 2021-10-01 12:55:39.497648
Doing this simulation: LOWBMUDDB at 2021-10-01 12:55:39.505940
Made dir: E:\GBRFScenarios\CSIROAggr

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node5_x'} in the result is deprecated and will raise a MergeError in a future version.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Outlet Node5_x'} in the result is deprecated and will raise a MergeError in a future version.


Done PLANFERTDB at 2021-10-01 12:58:37.781796
Doing this simulation: PLANMUDDB at 2021-10-01 12:58:37.791887
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\MackayWhitsundays\PLANMUDDB
Done PLANMUDDB at 2021-10-01 12:58:56.596647
Doing this simulation: PLANFERTCB at 2021-10-01 12:58:56.596647
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\MackayWhitsundays\PLANFERTCB
Done PLANFERTCB at 2021-10-01 12:59:15.649842
Doing this simulation: PLANMUDCB at 2021-10-01 12:59:15.651901
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\MackayWhitsundays\PLANMUDCB
Done PLANMUDCB at 2021-10-01 12:59:35.457292
Doing this simulation: BUBASE at 2021-10-01 12:59:35.457292
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\BUBASE
Done BUBASE at 2021-10-01 13:00:04.601681
Doing this simulation: BUPREDEV at 2021-10-01 13:00:04.601681
Made dir: E:\GBRFScenarios\CSIROAggregatedOutlets\GBRDynSedNet\Burdekin\BUPREDEV
Done BUPREDEV at 2021-10-01 13